# Intro to Crypto by Breaking Things

Let's start by defining the game in Python

In [ ]:
class SemanticSecurityGame:
    
    def __init__(self, cryptoScheme, challenger):
        self.cryptoScheme = cryptoScheme
        self.challenger = challenger
        self.numberOfTrials = 0
        self.numberOfSuccesses = 0

    def runGame(self, maximumTrials):
        from random import randint
        while self.numberOfTrials < maximumTrials:
            m1, m2 = self.challenger.getMessages(self.numberOfTrials)
            choice = randint(0, 1)

            plainText = m2
            if choice == 0:
                plainText = m1

            cipherText = self.cryptoScheme.encrypt(plainText)

            result = self.challenger.challenge(cipherText, self.numberOfTrials)

            if result == choice:
                self.numberOfSuccesses = self.numberOfSuccesses + 1
            
            self.numberOfTrials =  self.numberOfTrials + 1
        
        return { "Trials": self.numberOfTrials, "Wins" : self.numberOfSuccesses }
            

Next we want to implement RC4 as a crypto-scheme following the contract laid out here.  

The code for this follows below:

In [ ]:
class RC4:
   
    def __init__(self, key):
        self.internalState = bytearray(range(256))
        x=0
        for i in range(256):
            x = (x + self.internalState[i] + key[i % len(key)]) % 256
            self.swap(x, i)

        self.x = 0 
        self.y = 0

    def swap(self, x, y):
        self.internalState[x], self.internalState[y] = self.internalState[y], self.internalState[x]

    def clock(self):
         self.x = (self.x + 1) % 256
         self.y = (self.y + self.internalState[self.x]) % 256
         self.swap(self.x, self.y)

         internalPosition = (self.internalState[self.x] + self.internalState[self.y]) % 256

         return self.internalState[internalPosition]

    def encrypt(self, data):
        out = []
        for byte in data:
            out.append(byte ^ self.clock())
        return out

We can check we got the code right for the RC4 by running a check from RFC 6629:

In [ ]:
data = [0]*16
key = [0x01,0x02,0x03,0x04,0x05]

testVector = [0xb2,0x39,0x63,0x05,0xf0,0x3d,0xc0,0x27,0xcc,0xc3,0x52,0x4a,0x0a,0x11,0x18,0xa8] 

rc4 = RC4(key)

cipherText = rc4.encrypt(data)

cipherText == testVector, cipherText, testVector

## Using the same key twice with a stream cipher

Stream ciphers (generally) a new random key for each message that's encrypted. This is definitely the case with RC4. Let's show why re-using the key is insecure.

The first thing we need to do is create a crypto-scheme that just encrypts with the same key each time:

In [ ]:
class RC4RepeatedKey:
    def encrypt(self, data):
        rc4 = RC4([0, 1, 2, 3, 4])
        return rc4.encrypt(data)

Next, we need a strategy to defeat the game. In this case, we'll pass the same message twice on the first iteration, get the cipher text and use this to distinguish.

In [ ]:
class RC4RepeatedKeyStrategy:
    def getMessages(self, trialNumber):
        m0 = [0, 0, 0, 0]
        m1 = [1, 1, 1, 1]

        if trialNumber == 0:
            return m0, m0
        else:
            return m0, m1 

    def challenge(self, challenge, trialNumber):
        if trialNumber == 0:
            self.m0Encrypted = challenge
            return 0
        else:
            if challenge == self.m0Encrypted:
                return 0
            else:
                return 1

Now let's actually try our first challenge!!!!

In [ ]:
cipherScheme = RC4RepeatedKey()
strategy = RC4RepeatedKeyStrategy()
game = SemanticSecurityGame(cipherScheme, strategy)
game.runGame(200)

# Weird biases in RC4

RC4 got retired for a reason. It has some biases that make it distinguishable from a truly random source. 

We're going to look at one of these biases in particular. It turns out that the second byte generated from RC4 is zero twice as often as it should be.

Because there are 256 different values for a byte, ranging from 0 to 255, we'd expect to see the second byte be zero only 1/256th of the time. But instead, we see the frequency be 2/256th of the time.

Let's start by creating a new RC4 encryption scheme that randomly generates a new key for each message!


In [ ]:
class RC4RandomKey:
    def encrypt(self, data):
        rc4 = RC4(self.generateKey())
        return rc4.encrypt(data)

    def generateKey(self):
        from random import choice
        byteArray = range(255)
        keyMaterial = [choice(byteArray) for _ in range(16)]
        return keyMaterial

We also need a new strategy. Here we have a two byte message, one of all zeros and the other of all ones. 

We look at the second byte:

 - If it is zero, we say its m0. 
 - If it's 255 we say it's m1.

We need a lot of trials to reach statistical significance. 

Next we write a class to assess whether a certain number of coin flips is significant or not.

In [ ]:
def IsBias(heads, totalFlips):
    from math import sqrt 

    fairCoinHeadsProb = 0.5
    variancePerFlip = fairCoinHeadsProb*(1- fairCoinHeadsProb)
    expectedTotalVariance = variancePerFlip * totalFlips

    expectedNumberOfHeads = totalFlips * fairCoinHeadsProb
    expectedStandardDev = sqrt(expectedTotalVariance)

    highCutOff  = expectedNumberOfHeads + (3 * expectedStandardDev)
    lowCutOff = expectedNumberOfHeads - (3 * expectedStandardDev)

    return heads > highCutOff or heads < lowCutOff

Let's try it out and a few test cases:

In [ ]:
print(IsBias(199, 200))
print(IsBias(50, 100))
print(IsBias(60, 100))
print(IsBias(70, 100))
print(IsBias(20, 100))

Okay, let's build our strategy now to defeat RC4 using the bias on the second byte.

In [ ]:
class RC4BiasByteStrategy:
    def getMessages(self, trialNumber):
        m0 = [0, 0]
        m1 = [255, 255]

        return m0, m1 

    def challenge(self, challenge, trialNumber):
        if challenge[1] == 0:
            return 0
        elif challenge[1] == 255: 
           return 1
        else:
            return 0 # Here we just pick 0 arbitrarily. 

Any now we run our strategy and see how it goes!

In [ ]:
cipherScheme = RC4RandomKey()
strategy = RC4BiasByteStrategy()
game = SemanticSecurityGame(cipherScheme, strategy)
result = game.runGame(1000000)
print(result)

IsBias(result["Wins"], result["Trials"])